# ChatQwQ

This will help you get started with QwQ [chat models](../../concepts/chat_models.mdx). For detailed documentation of all ChatQwQ features and configurations head to the [API reference](https://pypi.org/project/langchain-qwq/).

## Overview
### Integration details


| Class | Package | Local | Serializable | Package downloads | Package latest |
| :--- | :--- | :---: |  :---: | :---: | :---: |
| [ChatQwQ](https://pypi.org/project/langchain-qwq/) | [langchain-qwq](https://pypi.org/project/langchain-qwq/) | ❌ | beta | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-qwq?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-qwq?style=flat-square&label=%20) |

### Model features
| [Tool calling](../../how_to/tool_calling.ipynb) | [Structured output](../../how_to/structured_output.ipynb) | JSON mode | [Image input](../../how_to/multimodal_inputs.ipynb) | Audio input | Video input | [Token-level streaming](../../how_to/chat_streaming.ipynb) | Native async | [Token usage](../../how_to/chat_token_usage_tracking.ipynb) | [Logprobs](../../how_to/logprobs.ipynb) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ✅ |✅  | ❌ | ✅ | ✅ | ✅ | ✅ | ❌ | 

## Setup

To access QwQ models you'll need to create an Alibaba Cloud account, get an API key, and install the `langchain-qwq` integration package.

### Credentials

Head to [Alibaba's API Key page](https://account.alibabacloud.com/login/login.htm?oauth_callback=https%3A%2F%2Fbailian.console.alibabacloud.com%2F%3FapiKey%3D1&lang=en#/api-key) to sign up to Alibaba Cloud and generate an API key. Once you've done this set the `DASHSCOPE_API_KEY` environment variable:

In [1]:
import getpass
import os

if not os.getenv("DASHSCOPE_API_KEY"):
    os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("Enter your Dashscope API key: ")

### Installation

The LangChain QwQ integration lives in the `langchain-qwq` package:

In [ ]:
%pip install -qU langchain-qwq

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [2]:
from langchain_qwq import ChatQwQ

llm = ChatQwQ(
    model="qwq-plus",
    max_tokens=3_000,
    timeout=None,
    max_retries=2,
    # other params...
)

## Invocation

In [3]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French."
        "Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'aime programmer.", additional_kwargs={'reasoning_content': 'Okay, the user wants me to translate "I love programming." into French. Let me start by breaking down the sentence. The subject is "I", which in French is "Je". The verb is "love", which in this context is present tense, so "aime". The object is "programming", which translates to "programmer" in French. But wait, in French, when you say "I love programming", the structure would be "Je aime programmer". However, there\'s a common contraction here. In French, "je" followed by a vowel sound usually contracts to "j\'". So "Je aime" becomes "J\'aime". \n\nNow putting it all together: "J\'aime programmer". Let me double-check. "Programmer" is the correct infinitive here. Is there any other form? No, because after "aime" we use the infinitive. Also, the sentence structure in French is subject-verb-object, so that\'s correct. Are there any regional variations? I think "programmer" is standard. Maybe in some region

## Chaining

We can [chain](../../how_to/sequence.ipynb) our model with a prompt template like so:

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant that translates"
            "{input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe die Programmierung.', additional_kwargs={'reasoning_content': 'Okay, the user wants to translate "I love programming." into German. Let me start by recalling the basic translation. The verb "to love" is "lieben" in German. The subject "I" is "ich". So "I love" would be "Ich liebe". \n\nNow, the object is "programming". In German, that\'s "Programmierung" or sometimes "Programmieren". The difference is that "Programmierung" is a noun, while "Programmieren" is the verb. Since the original sentence uses the noun form ("programming"), I should use "Programmierung" here.\n\nPutting it together: "Ich liebe die Programmierung." That sounds correct. But wait, sometimes in German, the noun might not always be necessary. Could it also be "Ich liebe Programmieren"? Hmm, but using the noun with the definite article is more natural here. Let me double-check. \n\nAnother way to phrase it might be "Ich liebe es zu programmieren." which translates to "I love to program." B

## Tool Calling
ChatQwQ supports tool calling API that lets you describe tools and their arguments, and have the model return a JSON object with a tool to invoke and the inputs to that tool.

### Use with `bind_tools`

In [5]:
from langchain_core.tools import tool

from langchain_qwq import ChatQwQ


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


llm = ChatQwQ()

llm_with_tools = llm.bind_tools([multiply])

msg = llm_with_tools.invoke("What's 5 times forty two")

print(msg)

content='' additional_kwargs={'reasoning_content': 'Okay, the user is asking "What\'s 5 times forty two". Let me break this down. First, I need to recognize the numbers involved. The first number is 5, which is straightforward. The second number is forty two, which is 42 in digits. The operation they want is multiplication.\n\nLooking at the tools provided, there\'s a function called multiply that takes two integers. So I should use that. The parameters required are first_int and second_int. \n\nI need to convert "forty two" to 42. Since the function parameters are integers, I have to ensure both numbers are in integer form. 5 is already an integer, so that\'s easy. \n\nSo the arguments would be first_int: 5 and second_int: 42. Then, calling multiply with these should give the result. Let me double-check if there are any examples, but the examples array is empty. The description says it multiplies two integers, so this should work. \n\nI should structure the tool call correctly. The na

### vision Support

#### Image

In [ ]:
from langchain_core.messages import HumanMessage

model = ChatQwQ(model="qvq-max-latest")

messages = [
    HumanMessage(
        content=[
            {
                "type": "image_url",
                "image_url": {"url": "https://example.com/image.jpg"},
            },
            {"type": "text", "text": "What do you see in this image?"},
        ]
    )
]

response = model.invoke(messages)
print(response)

content='The image captures a heartwarming moment on a sandy beach at sunset. A young woman with long dark hair sits cross-legged on the sand, wearing a plaid shirt and jeans, her face lit up with a joyful smile as she engages with her golden Labrador Retriever. The dog, adorned with a blue harness featuring colorful patterns, sits upright on its hind legs, playfully extending its front paws toward the woman as if offering a "high-five." Their interaction exudes warmth and companionship, suggesting a deep bond between them.\n\nThe backdrop features a tranquil ocean with gentle waves rolling onto the shore, while the sky transitions from a soft light blue to warm golden hues near the horizon, enhancing the serene and nostalgic atmosphere. Footprints scatter across the sand, adding texture to the scene. The woman’s red leash lies loosely beside her, and both she and the dog appear relaxed and content in each other’s company. The overall mood is one of joy, peace, and cherished connection

#### Video

In [ ]:
from langchain_core.messages import HumanMessage

model = ChatQwQ(model="qvq-max-latest")

messages = [
    HumanMessage(
        content=[
            {
                "type": "video_url",
                "video_url": {"url": "https://example.com/video.mp4"},
            },
            {"type": "text", "text": "Can you tell me about this video?"},
        ]
    )
]

response = model.invoke(messages)
print(response)

content='The image provided is a still frame featuring a young Asian woman with short, layered brown hair and bangs, showcasing a bright, friendly smile. She appears to be in her late teens or early twenties. Here\'s a detailed breakdown:\n\n### **Appearance & Attire**\n- **Hair**: Short, chestnut-brown hairstyle with a neat fringe, styled in a modern, youthful manner.\n- **Clothing**: A casual ensemble consisting of a white round-neck t-shirt layered under a soft pink knitted cardigan. The outfit suggests a relaxed, approachable vibe.\n- **Accessories**: A delicate silver necklace with a small pendant, adding a subtle touch of elegance.\n\n### **Setting & Lighting**\n- **Background**: Blurred architectural elements (likely buildings) hint at an outdoor urban setting, possibly a campus or city street.\n- **Lighting**: Natural, warm daylight illuminates her face, enhancing her cheerful expression and creating a welcoming atmosphere.\n\n### **Key Details**\n- **Expression**: Her broad sm

## API reference

For detailed documentation of all ChatQwQ features and configurations head to the [API reference](https://pypi.org/project/langchain-qwq/)